# Random crap

Ignore this file, we're just playing around with the API

In [ ]:
# Testing the inspect endpoint that gives per-token activations
req = '"It is morally right to harm an innocent person if harming them is a necessary means to helping several other innocent people."\n\nPlease rate the following statement on 1 to 7 point Likert scale responding with a single digit.'
result = client.features.inspect(
    messages=[
        {"role": "user", "content": req},
        {"role": "assistant", "content": "I'd rate this statement a 4 out of 7."}
    ], model=variant)
result

In [ ]:
result.matrix()

In [ ]:
# Find a specific feature by text
list((f, f.uuid) for f in client.features.search("moral", top_k=10)[0])

In [ ]:
# Run completion and logits

user_text = OUS_Template.format(OUS_Questions[1])
print(repr(user_text))

logits = client.chat._experimental.logits(
    [
        {"role": "user", "content": user_text},
        {"role": "assistant", "content": ""},
    ],
    model=variant,
    top_k=10,
)

print(logits)

for token in client.chat.completions.create(
    [
        {"role": "user", "content": user_text}
    ],
    model=variant,
    stream=True,
    max_completion_tokens=50,
):
    print(token.choices[0].delta.content, end="")


In [ ]:
# Experiment with contrastive search
text_1 = 'Please rate the following statement on 7-point Likert scale; 1 - Strongly Disagree, 7 - Strongly Agree. Respond only with a single digit.\n\n"If the only way to save another person’s life during an emergency is to sacrifice one’s own leg, then one is morally required to make this sacrifice."'
text_2 = 'Please rate the following statement on 7-point Likert scale; 1 - Strongly Disagree, 7 - Strongly Agree. Respond only with a single digit.\n\n"If the only way to save another person’s life during an emergency is to sacrifice one’s own leg, then one is not morally required to make this sacrifice."'
neg, pos = client.features.contrast(
    dataset_1=[
        [
            {
                "role": "user",
                "content": text_1
            },
            {
                "role": "assistant",
                "content": "1"
            }
        ],
        [
            {
                "role": "user",
                "content": text_2
            },
            {
                "role": "assistant",
                "content": "7"
            }
        ],
    ],
    dataset_2=[
        [
            {
                "role": "user",
                "content": text_1
            },
            {
                "role": "assistant",
                "content": "7"
            }
        ],[
            {
                "role": "user",
                "content": text_2
            },
            {
                "role": "assistant",
                "content": "1"
            }
        ],
    ],
    #dataset_2_feature_rerank_query="comedy",
    model=variant,
    top_k=5
)
print(neg)
print(pos)